In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: D:\ID2223 Scalable Machine Learning\Lab 1\mlfs-original
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
import importlib
from mlfs.airquality import util
importlib.reload(util)
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 10:28:34,138 INFO: Initializing external client
2025-11-18 10:28:34,141 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 10:28:37,072 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286306


'{"country": "sweden", "city": "stockholm", "street": "st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": "59.3251172", "longitude": "18.0710935"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_original',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_original',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
# aq_today_df = aq_today_df.drop(['country','city','street','url'],axis = 'columns')
aq_today_df

,pm25,country,city,street,date,url
0,16.0,sweden,stockholm,st-eriksgatan-83,2025-11-18,https://api.waqi.info/feed/@10523


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-1.70,0.0,7.594208,301.429504,stockholm
1,2025-11-19,-1.30,0.0,4.735060,351.253906,stockholm
2,2025-11-20,1.15,0.1,16.394829,8.841739,stockholm
3,2025-11-21,-3.35,0.1,15.575981,303.690094,stockholm
4,2025-11-22,1.60,0.0,15.463246,245.224884,stockholm
5,2025-11-23,3.45,0.1,10.144082,117.474350,stockholm
6,2025-11-24,2.80,0.0,11.269782,63.435013,stockholm
7,2025-11-25,-0.30,0.0,11.520000,360.000000,stockholm


In [8]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')
hourly_df

Coordinates 59.25°N 18.0°E
Elevation 23.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
date,,,,
2025-11-18 00:00:00,-3.55,0.0,11.709688,315.000092
2025-11-18 01:00:00,-3.85,0.0,10.464797,310.814972
2025-11-18 02:00:00,-4.20,0.0,9.585739,304.286926
2025-11-18 03:00:00,-4.45,0.0,8.854829,296.564972
2025-11-18 04:00:00,-4.65,0.0,8.427383,289.983185
...,...,...,...,...
2025-11-25 19:00:00,-1.55,0.0,10.308831,12.094739
2025-11-25 20:00:00,-1.50,0.0,10.661107,11.689341
2025-11-25 21:00:00,-1.50,0.0,11.013882,11.309896


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
csv_file=f"{root_dir}/data/air-quality-data.csv"
util.check_file_path(csv_file)
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
latest_day = df['date'].max()
# latest_day = pd.to_datetime(latest_day)  # Keep it timezone-naive
latest_day = pd.to_datetime(latest_day)
latest_day

File successfully found at the path: D:\ID2223 Scalable Machine Learning\Lab 1\mlfs-original/data/air-quality-data.csv


Timestamp('2025-11-17 00:00:00')

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 10:28:41,913 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286306/fs/1273933/fg/1730911


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_original_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286306/jobs/named/air_quality_original_1_offline_fg_materialization/executions


(Job('air_quality_original_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 766995
         }
       },
       "result": {
         "observed_value": 16.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:28:41.000913Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful

In [11]:
# air_quality_fg.read().set_index('date').sort_index()
air_quality_fg.read().set_index('date').sort_index()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


,pm25,country,city,street,url
date,,,,,
2018-04-09 00:00:00+00:00,55.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-10 00:00:00+00:00,14.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-11 00:00:00+00:00,23.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-12 00:00:00+00:00,26.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2018-04-13 00:00:00+00:00,33.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
...,...,...,...,...,...
2025-11-14 00:00:00+00:00,13.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2025-11-15 00:00:00+00:00,12.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2025-11-16 00:00:00+00:00,15.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523


In [12]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 10:29:15,359 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286306/fs/1273933/fg/1730912


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 8/8 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_original_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286306/jobs/named/weather_original_1_offline_fg_materialization/executions
2025-11-18 10:29:31,751 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 10:29:34,941 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 10:31:45,485 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 10:31:45,643 INFO: Waiting for log aggregation to finish.
2025-11-18 10:32:04,206 INFO: Execution finished successfully.


(Job('weather_original_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 766997
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 8,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:29:15.000358Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expec

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
